In [1]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict

/Users/eviemcgonigle/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/eviemcgonigle/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#for pos tagging

#!pip install spacy
import spacy

!python -m spacy download en_core_web_sm
en_spacy = spacy.load('en_core_web_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import re #regex for cleaning


In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/eviemcgonigle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/eviemcgonigle/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eviemcgonigle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/eviemcgonigle/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# IMPORT DATA 

In [5]:
def import_posts(filename):
    data = []
    with open(filename, 'r') as file:
        for json_str in file:
        # load lines
            result = json.loads(json_str)

            # extract fields
            data.append({
                'selftext': result.get('selftext', None),
                'title': result.get('title', None),
                'author': result.get('author', None)
            })

    df = pd.DataFrame(data)
    return df

In [6]:
def import_comments(filename):
    data = []
    with open(filename, 'r') as file:
        for json_str in file:
        # load lines
            result = json.loads(json_str)

            # extract fields
            data.append({
                'author': result.get('author', None),
                'selftext': result.get('body', None)
            })
    df = pd.DataFrame(data)
    
    #add in empty column 'title' to use concat function
    df['title'] = np.nan
    return df


In [7]:
#remove posts from known bots
#and posts where content is removed
def remove_unhelpful(data):
    df = data.copy()
    
    #remove bots
    bot_authors = ["AutoModerator", "ClickableLinkBot", "election_info_bot", 
                   "outline_link_bot", "LearnDifferenceBot"]
    df = df[~df['author'].isin(bot_authors)]
    
    
    #remove empty deleted posts/comments
    df = df[~(((df['selftext'] == "[removed]")|(df['selftext'] == "[deleted]")) 
              & (df['title'].isna() | (df['title'] == "") | (df['title'] == "[deleted by user]")))]
    
    return df

# Random Downsampling
# make each df the length of the smallest df
# with same number of unique authors
def rand_downsampling(d23, d19, r23, r19, rstate):
    
    #first make each dataset have the same number of unique authors
    unique_authors_counts = [
        d23['author'].nunique(),
        d19['author'].nunique(),
        r23['author'].nunique(),
        r19['author'].nunique()
    ]
    min_authors = min(unique_authors_counts)
    
    #same namber unique authors per corpus
    def downsample_author(df, min_authors, rstate):
        sampled_authors = df['author'].drop_duplicates().sample(n=min_authors, random_state=rstate)
        return df[df['author'].isin(sampled_authors)]
    
    d23_down = downsample_author(d23, min_authors, rstate)
    d19_down = downsample_author(d19, min_authors, rstate)
    r23_down = downsample_author(r23, min_authors, rstate)
    r19_down = downsample_author(r19, min_authors, rstate)
    
    lengths = [len(d23_down), len(d19_down), len(r23_down), len(r19_down)]
    size = min(lengths)
    
    final_down_d23 = d23_down.sample(n=size, random_state=rstate).reset_index(drop=True)
    final_down_d19 = d19_down.sample(n=size, random_state=rstate).reset_index(drop=True)
    final_down_r23 = r23_down.sample(n=size, random_state=rstate).reset_index(drop=True)
    final_down_r19 = r19_down.sample(n=size, random_state=rstate).reset_index(drop=True)
    
    return final_down_d23, final_down_d19, final_down_r23, final_down_r19

#combine post and comment data into one df
def combo(posts, comments):
    return pd.concat([posts, comments], ignore_index=True)

In [8]:
#TESTERS

def unique_auth(d23, d19, r23, r19):
    unique_authors_counts = [
        d23['author'].nunique(),
        d19['author'].nunique(),
        r23['author'].nunique(),
        r19['author'].nunique()
    ]
    return unique_authors_counts

def unique_auth_posts(d23, d19, r23, r19, rstate):
    unique_authors_counts = [
        d23['author'].nunique(),
        d19['author'].nunique(),
        r23['author'].nunique(),
        r19['author'].nunique()
    ]
    min_authors = min(unique_authors_counts)
    
    def downsample_by_authors(df, min_authors, rstate):
        sampled_authors = df['author'].drop_duplicates().sample(n=min_authors, random_state=rstate)
        return df[df['author'].isin(sampled_authors)]
    
    d23_down = downsample_by_authors(d23, min_authors, rstate)
    d19_down = downsample_by_authors(d19, min_authors, rstate)
    r23_down = downsample_by_authors(r23, min_authors, rstate)
    r19_down = downsample_by_authors(r19, min_authors, rstate)
    return d23_down, d19_down, r23_down, r19_down

def remove_del(data):
    df = data.copy()
    df = df[~((df['selftext'] == "[removed]") & (df['title'].isna() | (df['title'] == "")))]
    return df

In [9]:
#Import Posts

p_d23 = remove_unhelpful(import_posts('./data/raw/r_democrats_23posts.jsonl'))
p_d19 = remove_unhelpful(import_posts('./data/raw/r_democrats_19posts.jsonl'))
p_r23 = remove_unhelpful(import_posts('./data/raw/r_republican_23posts.jsonl'))
p_r19 = remove_unhelpful(import_posts('./data/raw/r_republican_19posts.jsonl'))

#downsample
down_p_d23, down_p_d19, down_p_r23, down_p_r19 = rand_downsampling(p_d23, p_d19, p_r23, p_r19, 3)

print(len(down_p_d23), len(down_p_d19), len(down_p_r23), len(down_p_r19))

8766 8766 8766 8766


In [10]:
#Import Comments
c_d23 = remove_unhelpful(import_comments('./data/raw/r_democrats_23comments.jsonl'))
c_d19 = remove_unhelpful(import_comments('./data/raw/r_democrats_19comments.jsonl'))
c_r23 = remove_unhelpful(import_comments('./data/raw/r_republican_23comments.jsonl'))
c_r19 = remove_unhelpful(import_comments('./data/raw/r_republican_19comments.jsonl'))

#downsample
down_c_d23, down_c_d19, down_c_r23, down_c_r19 = rand_downsampling(c_d23, 
                                                                   c_d19, c_r23, c_r19, 10)

print(len(down_c_d23), len(down_c_d19), len(down_c_r23), len(down_c_r19))

20481 20481 20481 20481


In [12]:
# concatenate posts and comments for each dataset
data_d23 = combo(down_p_d23, down_c_d23)
data_d19 = combo(down_p_d19, down_c_d19)
data_r19 = combo(down_p_r19, down_c_r19)
data_r23 = combo(down_p_r23, down_c_r23)


In [14]:
#check print(len(data_d23), len(data_d19), len(data_r23), len(data_r19))

29247 29247 29247 29247


In [15]:
#save data
data_d23.to_csv('./data/raw/r_democrats_23down_auth.csv', index = False)
data_d19.to_csv('./data/raw/r_democrats_19down_auth.csv', index = False)
data_r23.to_csv('./data/raw/r_republican_23down_auth.csv', index = False)
data_r19.to_csv('./data/raw/r_republican_19down_auth.csv', index = False)

# Cleaning Functions

In [17]:
def concat(df):
    #remove deleted/removed from selftext (no added meaning)
    df['selftext'] = df['selftext'].replace('[removed]', '', regex=False)
    df['selftext'] = df['selftext'].replace('[deleted]', '', regex=False)
    
    df['selftext'] = df['selftext'].fillna('').astype(str)
    df['title'] = df['title'].fillna('').astype(str)

    df['text'] = df['selftext'] + " " + df['title']
    return df[['author', 'text']].copy()
        


In [18]:
#based off evolving divergence paper
#removes long posts that are duplicates in text
#does NOT remove reprtive Urls - exmained and they are news articles etc
def filter_spam(data):
    #remove duplicated posts
    #df = df[(df['text'].str.len() <= 100) | 
    #        ((df['text'].str.len() > 100) & 
    #         ~df.duplicated(subset=['text']))].copy()
    
    df = data.copy()
    
    #remove bots
    bot_patterns = [
        r"beep,? boop,? i'?m a bot", 
        r"i am a bot",  
        r"this action was performed automatically",  
        r"please contact the moderators",  
        r"source code for this bot",  
        r"this response was generated",
        r"Bot message:"
    ]
    bot_regex = "|".join(bot_patterns)  # Join patterns into a single regex
    df = df[~df['text'].str.contains(bot_regex, case=False, na=False, regex=True)]
    
#     df.loc[:, 'url'] = df['text'].str.extract(r"(https://\S+)")
#     spam_urls = df['url'].value_counts().nlargest(30).index
#     df['is_spam'] = df['url'].isin(spam_urls) 
    
    #remove user/subreddit mentions
    df['text'] = df['text'].str.replace(r"(/r/\w+)", " ", regex=True)
    df['text'] = df['text'].str.replace(r"(/u/\w+)", " ", regex=True)
    
    #remove urls
    df['text'] = df['text'].str.replace(r"http[s]?://\S+", " ", regex=True)
    
    #extra spaces
    df['text'] = df['text'].str.replace(r"\s+", " ", regex=True)\
        .str.strip()          #leading/trailing spaces
    
    df['text'] = df['text'].fillna('').astype(str)
    return df
        
 

In [19]:
#create new df with sentences, authors instead of text, authors
#runs on cleaned text
def to_sentences(row):
    author = row['author']
    text = row['clean_text']
    post_id = row.name
    
    if not isinstance(text, str):
        print(f"Non-string value found: {text} (type: {type(text)})")
        return []
    
    all_sent = sent_tokenize(text)

    df_sent = [{'author': author, 'sentence': sentence, 'post_id': post_id} for sentence in all_sent]
    
    return df_sent

def text_cleaner(post):
    
    post = post.lower()  #lower case all text
    
    post = re.sub(r'[^a-zA-Z0-9\s.!?]', '', post) #remove special char (not punctuation)
    
    post = re.sub(r"(?<!\.)\n", ". ", post)  # line breaks to sentence breaks
    post = re.sub(r"([a-zA-Z0-9])\n", r"\1. ", post)  # make sentences end properly
    
    #remove 0 width space character
    post = re.sub('\x200b', '', post)
    
    #quo thing
    post = re.sub('quid pro quo', 'quidproquo', post)
    post = re.sub('status quo', 'statusquo', post)
    post = re.sub('dems', 'democrats', post)
    
    #name processing
    post = re.sub(r"(\bgov|mr|ms|dr|jr)\.", r"\1", post)
    post = re.sub(r"\b(?:realdonaldtrump|donald j\. trump)", "donald trump", post)
    
    names_places = (
        r'\b('
        r'donald trump|kamala harris|joe biden|hunter biden|bernie sanders|barack obama|michelle obama|'
        r'elizabeth warren|mitch mcconnell|alex jones|nancy pelosi|betsy devos|robert mueller|ron desantis|'
        r'ilhan omar|marjorie taylor greene|clarence thomas|kevin mccarthy|chris christie|pete buttigieg|'
        r'el paso|new york|white house|new york city|new hampshire|north dakota|south dakota|'
        r'north carolina|south carolina|rhode island|new mexico|new jersey|west virginia'
        r')'
        r'(?=\b|[.!?]\s|$)'
    )
    post = re.sub(names_places, lambda x: x.group().replace(" ", ""), post)

    #politicians more commonly known by 1 word 
    post = re.sub('alexandria ocasio cortez', 'aoc', post)
    post = re.sub('beto orourke', 'beto', post)
    
    #post = re.sub('', '', post)
    return post

In [20]:
#removes stop words, and lemmatizes
def lemma_er(post):
    
    stop_words = set(stopwords.words('english'))
    post = [word for word in post if word not in stop_words]
    
    post = [lemmatizer.lemmatize(word) for word in post]
    
    return post

#tag parts of speech (mostly for PPN ID)
def pos_tagger(tokens):
    doc = en_spacy(" ".join(tokens))  
    return [(token.text, token.pos_) for token in doc]  

In [21]:
#main functions to actually call
#filter spam, preprocess, split text into sentences  
def get_sentences(df):
    data = concat(df)

    data = filter_spam(data).copy()
    
    #clean text
    data['clean_text'] = data['text'].apply(text_cleaner)

    #split text into sentences - nec for fasttext
    sentence_data = data.apply(to_sentences, axis=1)
    flat_sentences = [item for sublist in sentence_data for item in sublist]
    
    return pd.DataFrame(flat_sentences)

#returns sentences split into words and author
def get_words_nltk(sent):
    sents = sent.copy()
    
    #remove punctuation
    sents['sentence'] = sents['sentence'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

    #tokenize words 
    sents['tokens'] = sents['sentence'].apply(lambda x: word_tokenize(x) if x.strip() else [])

    #remove empty token rows
    sents = sents[sents['tokens'].apply(lambda x: len(x) > 0)]
    
    #tag parts of speech
    sents['pos_tags'] = sents['tokens'].apply(pos_tagger)
    
    #lemmatize
    sents.loc[:, 'out_sent'] = sents['tokens'].apply(lemma_er)
    
    return sents

#batch process sentences with spacy
def get_words(sent):
    sents = sent.copy()
    
    docs = list(en_spacy.pipe(sents['sentence'].tolist()))
    
    sents['tokens'] = [[token.text for token in doc] for doc in docs]
    sents['pos_tags'] = [[(token.text, token.pos_) for token in doc] for doc in docs]
    sents['out_sent'] = [[token.lemma_ for token in doc] for doc in docs]
    
    #remove empties
    sents = sents[sents['tokens'].apply(len) > 0]
    
    return sents
    

# RUN MAIN
change to run on combo data

In [22]:
data_d19 = pd.read_csv('./data/raw/r_democrats_19down_auth.csv')
data_d23 = pd.read_csv('./data/raw/r_democrats_23down_auth.csv')
data_r23 = pd.read_csv('./data/raw/r_republican_23down_auth.csv')
data_r19 = pd.read_csv('./data/raw/r_republican_19down_auth.csv')


In [24]:
sents_r19 = get_sentences(data_r19)
words_r19 = get_words(sents_r19)

sents_r23 = get_sentences(data_r23)
words_r23 = get_words(sents_r23)


In [25]:
words_r23.to_csv('./data/lemma/r_republican_23_words.csv', index = False)
words_r19.to_csv('./data/lemma/r_republican_19_words.csv', index = False)

In [26]:
sents_d23 = get_sentences(data_d23)
words_d23 = get_words(sents_d23)

sents_d19 = get_sentences(data_d19)
words_d19 = get_words(sents_d19)

words_d23.to_csv('./data/lemma/r_democrats_23_words.csv', index = False)
words_d19.to_csv('./data/lemma/r_democrats_19_words.csv', index = False)

In [23]:
#check empties

# data_filtered = filter_spam(concat((data_r23)))

# data_filtered['clean_text'] = data_filtered['text'].apply(text_cleaner)
# print("Posts that became empty after cleaning:", (data_filtered['clean_text'] == "").sum())

# empty_sentences = data_filtered['clean_text'].apply(lambda x: len(sent_tokenize(x)) == 0).sum()
# print("Posts that resulted in no sentences:", empty_sentences)


Posts that became empty after cleaning: 133
Posts that resulted in no sentences: 138


# POS TAG
dictionary
will do analysis on NNP(S) separately


In [28]:
def get_POS(df):
    #creat dictionary
    pos_dict = defaultdict(set)

    for pos_list in df['pos_tags']:
        for word, tag in pos_list:
            pos_dict[word].add(tag)

    #convert to df
    df_word_pos = pd.DataFrame(list(pos_dict.items()), columns=['word', 'tags'])
    
    return df_word_pos


In [29]:
pos_r19 = get_POS(words_r19)
pos_r19.to_csv('./data/lemma/r_republican_19_pos.csv', index = False)

pos_d19 = get_POS(words_d19)
pos_d19.to_csv('./data/lemma/r_democrats_19_pos.csv', index = False)

pos_r23 = get_POS(words_r23)
pos_r23.to_csv('./data/lemma/r_republican_23_pos.csv', index = False)

pos_d23 = get_POS(words_d23)
pos_d23.to_csv('./data/lemma/r_democrats_23_pos.csv', index = False)
#SAVE DICTS FOR ANALYSIS

In [33]:
print(len(pos_d23), len(pos_d19), len(pos_r23), len(pos_r19))

29188 30428 27460 29575


# TTR

In [38]:
#compute token-text ratio for corpus
#input: words_ df
def ttr(df):
    #flatten
    all_tokens = [token for tokens in df['tokens'] for token in tokens]
    unique_tok = set(all_tokens)

    return len(unique_tok) / len(all_tokens)

#compute for each corpus
print("TTR for D23:", ttr(words_d23))
print("TTR for D19:", ttr(words_d19))
print("TTR for R23:", ttr(words_r23))
print("TTR for R19:", ttr(words_r19))

TTR for D23: 0.038347237732378636
TTR for D19: 0.03244635815541579
TTR for R23: 0.035267705775375795
TTR for R19: 0.033832363071875614
